## Name: Luke Pratley
## Date Week 6

The purpose of this notebook is to see if the road dataset has overlap with the building dataset. Then we can create road masks that compliment the building foot print mask. The road dataset covers more area, so we expect only a subset of roads to be present in both datasets.

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import glob
import os

import geopandas as gpd
import rasterio
import rasterio.features
from rasterio.merge import merge
from rasterio.plot import show

import sys
sys.path.insert(0,os.path.abspath('../../building_road_segmentation'))

import building_road_segmentation


from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

In [49]:
data_directory = 'D:\Capstone Project\data\\'

In [50]:
datasets = glob.glob(data_directory + '*\\')
datasets = [d.split('\\')[-2] for d in datasets]
datasets

['AOI_2_Vegas',
 'AOI_2_Vegas_Train',
 'AOI_3_Paris',
 'AOI_3_Paris_Train',
 'AOI_4_Shanghai',
 'AOI_4_Shanghai_Train',
 'AOI_5_Khartoum',
 'AOI_5_Khartoum_Train',
 'final_images',
 'final_masks']

Below I am listing the directories in the dataset. 

In [100]:
directories_dict = building_road_segmentation.get_directories_dictionary(data_directory, 6)
directories_dict

{'MS': 'D:\\Capstone Project\\data\\AOI_5_Khartoum\\MS',
 'PAN': 'D:\\Capstone Project\\data\\AOI_5_Khartoum\\PAN',
 'PS-MS': 'D:\\Capstone Project\\data\\AOI_5_Khartoum\\PS-MS',
 'PS-RGB': 'D:\\Capstone Project\\data\\AOI_5_Khartoum\\PS-RGB',
 'building_mask': 'D:\\Capstone Project\\data\\AOI_5_Khartoum\\building_mask',
 'geojson_roads': 'D:\\Capstone Project\\data\\AOI_5_Khartoum\\geojson_roads',
 'road_mask': 'D:\\Capstone Project\\data\\AOI_5_Khartoum\\road_mask'}

This dataset doesn't have a summary table like the buildings one did.

## Data Cleaning

Here we are reading in information about the labels; the table containing the building locations:

In [101]:
geojson_files = [gpd.read_file(k) for k in glob.glob(directories_dict['geojson_roads'] + '\*')]

We can take a quick look at a file:

In [102]:
geojson_files[1]

,bridge_type,heading,lane_numbe,lane_number,one_way_type,paved,road_id,road_type,origarea,origlen,partialDec,truncated,geometry
0,2,0,2,2,2,1,7368,4,0,0.000397,1,0,"LINESTRING (32.53411 15.63126, 32.53415 15.63165)"
1,2,0,1,1,2,2,8213,5,0,0.000365,1,0,"LINESTRING (32.53343 15.63132, 32.53345 15.63168)"
2,2,0,2,2,2,2,10384,4,0,0.000360,1,0,"LINESTRING (32.53281 15.63137, 32.53283 15.631..."
3,2,0,2,2,2,2,10483,5,0,0.000700,1,0,"LINESTRING (32.53415 15.63165, 32.53345 15.63168)"
4,2,0,2,2,2,2,10737,5,0,0.000602,1,0,"LINESTRING (32.53345 15.63168, 32.53311 15.631..."
5,2,0,2,2,2,1,5146,4,0,0.000396,1,0,"LINESTRING (32.53417 15.63205, 32.53415 15.63165)"
6,2,0,2,2,2,2,6114,5,0,0.000405,1,0,"LINESTRING (32.53347 15.63208, 32.53345 15.63168)"
7,2,0,2,2,2,2,8151,4,0,0.000429,1,0,"LINESTRING (32.53289 15.63215, 32.53286 15.631..."
8,2,0,2,2,2,2,6376,5,0,0.000665,1,0,"LINESTRING (32.53417 15.63205, 32.53351 15.63208)"
9,2,0,2,2,2,2,8429,5,0,0.000036,1,0,"LINESTRING (32.53351 15.63208, 32.53347 15.63208)"


We can see information about the types of roads!

One major issue is that we don't have any way to inspect the image ids. We will have to build up our image ids manually.

Below we concatenate all of the files:

In [103]:
road_df = pd.concat(geojson_files, axis=0, ignore_index=True)

In [104]:
road_df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 10103 entries, 0 to 10102
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   bridge_type   10103 non-null  object  
 1   heading       10103 non-null  object  
 2   lane_numbe    10103 non-null  object  
 3   lane_number   10103 non-null  object  
 4   one_way_type  10103 non-null  object  
 5   paved         10103 non-null  object  
 6   road_id       10103 non-null  int64   
 7   road_type     10103 non-null  object  
 8   origarea      10103 non-null  int64   
 9   origlen       10103 non-null  float64 
 10  partialDec    10103 non-null  int64   
 11  truncated     10103 non-null  int64   
 12  geometry      10103 non-null  geometry
dtypes: float64(1), geometry(1), int64(4), object(7)
memory usage: 1.0+ MB


In [105]:
road_df.head()

,bridge_type,heading,lane_numbe,lane_number,one_way_type,paved,road_id,road_type,origarea,origlen,partialDec,truncated,geometry
0,2,0,2,2,2,1,8013,2,0,0.000771,1,0,"LINESTRING (32.48751 15.51732, 32.48744 15.51703)"
1,2,0,2,2,2,2,10836,5,0,0.000423,1,0,"LINESTRING (32.48901 15.51732, 32.48901 15.51732)"
2,2,0,2,2,2,2,11989,5,0,0.000362,1,0,"LINESTRING (32.48821 15.51625, 32.48815 15.51661)"
3,2,0,2,2,2,2,6562,5,0,0.000837,1,0,"LINESTRING (32.48815 15.51661, 32.48784 15.516..."
4,2,0,2,2,2,2,6731,5,0,0.000436,1,0,"LINESTRING (32.48901 15.51683, 32.48898 15.51640)"


There are no nulls in any of the columns. We find that there are duplicates for the road Ids.

In [106]:
road_df['road_id'].duplicated().sum()

1134

In [107]:
len(road_df['road_id'].unique())

8969

There are 16k roads

There is a non-truncated version of each of these roads.

In [108]:
road_df['road_id'].shape[0]

10103

Solving for the unique road gemoetry rows and keeping only the first:

In [109]:
unique_roads = road_df['road_id']

In [110]:
unique_roads

0         8013
1        10836
2        11989
3         6562
4         6731
         ...  
10098     3253
10099      106
10100    12681
10101     1692
10102    10624
Name: road_id, Length: 10103, dtype: int64

In [111]:
final_road_df = road_df.iloc[unique_roads.index, :]

In [112]:
final_road_df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 10103 entries, 0 to 10102
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   bridge_type   10103 non-null  object  
 1   heading       10103 non-null  object  
 2   lane_numbe    10103 non-null  object  
 3   lane_number   10103 non-null  object  
 4   one_way_type  10103 non-null  object  
 5   paved         10103 non-null  object  
 6   road_id       10103 non-null  int64   
 7   road_type     10103 non-null  object  
 8   origarea      10103 non-null  int64   
 9   origlen       10103 non-null  float64 
 10  partialDec    10103 non-null  int64   
 11  truncated     10103 non-null  int64   
 12  geometry      10103 non-null  geometry
dtypes: float64(1), geometry(1), int64(4), object(7)
memory usage: 1.1+ MB


In [113]:
print(final_road_df['geometry'][0])

LINESTRING (32.48751129243152 15.517321199936934, 32.487438558 15.517031424)


In [114]:

final_road_df.to_file(data_directory + datasets[7] + '\\summaryData\\roads.geojson', driver='GeoJSON')

In [115]:
buildings_geojson_files = [gpd.read_file(d) for d in glob.glob(data_directory + datasets[7] + '\\geojson\\buildings\\buildings_*.geojson')]
#building_geojsons = [d for d]


In [116]:
buildings_df = pd.concat(buildings_geojson_files, axis=0, ignore_index=True)
buildings_df.shape

(25463, 3)

In [117]:
buildings_df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 25463 entries, 0 to 25462
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   ImageId     25463 non-null  object  
 1   BuildingId  25463 non-null  int64   
 2   geometry    25463 non-null  geometry
dtypes: geometry(1), int64(1), object(1)
memory usage: 596.9+ KB


In [118]:
buildings_df

,ImageId,BuildingId,geometry
0,AOI_5_Khartoum_img1,1,"POLYGON Z ((32.48739 15.51458 0.00000, 32.4873..."
1,AOI_5_Khartoum_img1,2,"POLYGON Z ((32.48711 15.51460 0.00000, 32.4871..."
2,AOI_5_Khartoum_img1,3,"POLYGON Z ((32.48727 15.51460 0.00000, 32.4872..."
3,AOI_5_Khartoum_img1,4,"POLYGON Z ((32.48737 15.51471 0.00000, 32.4873..."
4,AOI_5_Khartoum_img1,5,"POLYGON Z ((32.48719 15.51472 0.00000, 32.4871..."
...,...,...,...
25458,AOI_5_Khartoum_img994,6,"POLYGON Z ((32.55597 15.56367 0.00000, 32.5559..."
25459,AOI_5_Khartoum_img994,7,"POLYGON Z ((32.55594 15.56384 0.00000, 32.5558..."
25460,AOI_5_Khartoum_img994,8,"POLYGON Z ((32.55581 15.56394 0.00000, 32.5557..."
25461,AOI_5_Khartoum_img994,9,"POLYGON Z ((32.55592 15.56400 0.00000, 32.5558..."


## Getting image names

In [119]:
road_image_names = [d.split('train_')[-1].replace('.tif', '') for d in glob.glob(directories_dict['PS-RGB'] + '\\*')]

In [120]:
len(road_image_names)

283

In [123]:
from shapely.geometry import box
key ='MS'
images_to_keep = []
for images in glob.glob(directories_dict['PS-RGB'] + '\\*'):
    with rasterio.open(images) as image:
        image_name = images.split('\\')[-1].replace('.tif', '')
        bounds = box(*image.bounds)
        check_bounds =  np.vectorize(lambda x : x.intersects(bounds))
        building_overlap = check_bounds(buildings_df['geometry'])
        road_overlap = check_bounds(final_road_df['geometry'])
        img = image.read()
        img = np.moveaxis(img, 0, 2)
        img = img/1800
        if road_overlap.sum() > 0:
            road_mask = rasterio.features.geometry_mask(final_road_df['geometry'][road_overlap], image.shape, image.transform, all_touched=False, invert=True)
        else:
            road_mask = -1 * np.ones((img.shape[0], img.shape[1]))
            
        if building_overlap.sum() > 0:
            building_mask = rasterio.features.geometry_mask(buildings_df['geometry'][building_overlap], image.shape, image.transform, all_touched=False, invert=True)
        else:
            building_mask = -1 * np.ones((img.shape[0], img.shape[1]))
        
        patch_size= (int(img.shape[0]/2), int(img.shape[1]/2))
        patches = [img[:(patch_size[0]),:(patch_size[1])], 
                   img[patch_size[0]:(2 * patch_size[0]),:(patch_size[1])], 
                   img[patch_size[0]:(2 * patch_size[0]), patch_size[1]:(2 * patch_size[1])],
                   img[:(patch_size[0]), patch_size[1]:(2 * patch_size[1])],
                  ]
        road_mask_patches = [road_mask[:(patch_size[0]),:(patch_size[1])], 
                   road_mask[patch_size[0]:(2 * patch_size[0]),:(patch_size[1])], 
                   road_mask[patch_size[0]:(2 * patch_size[0]), patch_size[1]:(2 * patch_size[1])],
                   road_mask[:(patch_size[0]), patch_size[1]:(2 * patch_size[1])]
                  ]
        building_mask_patches = [building_mask[:(patch_size[0]),:(patch_size[1])], 
                   building_mask[patch_size[0]:(2 * patch_size[0]),:(patch_size[1])], 
                   building_mask[patch_size[0]:(2 * patch_size[0]), patch_size[1]:(2 * patch_size[1])],
                   building_mask[:(patch_size[0]), patch_size[1]:(2 * patch_size[1])]
                  ]
        for k, a in enumerate(['a', 'b', 'c', 'd']):
            path = f'D:\\Capstone Project\\data\\final_masks\\road_mask_{image_name}_{a}.npy'
            np.save(path, road_mask_patches[k])
        for k, a in enumerate(['a', 'b', 'c', 'd']):
            path = f'D:\\Capstone Project\\data\\final_masks\\building_mask_{image_name}_{a}.npy'
            np.save(path, building_mask_patches[k])
        for k, a in enumerate(['a', 'b', 'c', 'd']):
            path = f'D:\\Capstone Project\\data\\final_images\\{image_name}_{a}.npy'
            np.save(path, patches[k])